# Εργασία στο Μάθημα "Ασφάλεια στο Διαδίκτυο"

Για την προσομοιώσεις ανταλλαγής κλειδιών DH και για της Επίθεσης Ενδιάμεσης Οντότητας, πέρα από τις συναρτήσεις που ζητούνται, υλοποιήθηκε και μια κλάση *Agent*, για την καλύτερη αναπαράσταση των οντοτήτων που λαμβάνουν μέρος.

## Άσκηση 1

Αρχικά υλοποιούνται οι συναρτήσεις που ζητούνται με τη βοήθεια του κώδικα από το LAB. 

In [1]:
def generate_parameters(bits):
    p=1
    while not is_prime(p):
        q=1
        while (q<2^(bits-2)):
            q=random_prime(2^bits)
        p=2*q+1
   
    F = GF(p)
    Zp = IntegerModRing(p)
    g=F(2)
    if multiplicative_order(Zp(g))!=(p-1):
        g=F(Mod(-2,p))

    return p,q,g,F

def public_secret_pair(p,q,g,F):
    x = F(randint(2,p-1))
    X = g ^ x
    return X,x

def generate_secret(X,y):
    return X ^ y


Έπειτα, ορίζουμε μια κλάση *Agent*, η οποία αφορά μια οντότητα. Οι παράμετροί της (p,q,g,F) εισάγωνται από ένα Output της συνάρτησης *generate_parameters(bits)* και υπολογίζονται σύμφωνα με αυτά τα κλειδιά της οντότητας, με χρήση της συνάρτησης *public_secret_pair(p,q,g,F)*. Τέλος υπάρχει μια επιπλέον μέθοδος, η *generate_session(self,agent_name,agent_public_key)* η οποία έχει αντίστοιχη λειτουργία με τη *generate_secret(X,y)*, όμως κάθε οντότητα κρατάει ξεχωριστά σε δική της λίστα τα sessions που εγκαθιδρύει (κρατάει οντότητα με την οποία έχει δημιουργηθεί συνεδρία και το κοινό μυστικό τους).

In [2]:
class Agent:
    sessions=[]

    def __init__(self,name,p,q,g,F):
        self.name = name
        self.p,self.q,self.g,self.F= p,q,g,F
        self.public_key, self.private_key = public_secret_pair(p,q,g,F)
        self.sessions=[]

    def generate_session(self,agent_name,agent_public_key):
        self.sessions.append([agent_name,agent_public_key^self.private_key])
        return

Η *key_exchange_simulation(bitsize)* παρουσιάζει τη διαδικασία ανταλλαγής κλειδιού μεταξύ της Alice και του Bob:

In [3]:

def key_exchange_simulation(bitsize): 


    # parameters
    p,q,g,F = generate_parameters(bitsize)
    print( "Using",bitsize,"bits for p:, p is",p,", q is",q,", generator g is ",g," and ",F )

    
    # Alice's side, sends A to Bob
    alice = Agent("Alice",p,q,g,F)
    print(alice.name,"'s secret key is",alice.private_key," calculates and sends theirs public key:",alice.public_key)
    A = alice.public_key

    # Bob's side, sends B to Alice
    bob = Agent("Bob",p,q,g,F)
    print(bob.name,"'s secret key b is",bob.private_key," calculates and sends theirs public key:",bob.public_key)
    B = bob.public_key

    #Calculate common secret
        #Alice, knows A,a and B
    alice.generate_session("Bob",B)
        #Bob, knows B,b and A
    bob.generate_session("Alice",A)

    print("Is the secret that both sides calculate the same value?",alice.sessions[0][1] == bob.sessions[0][1])

    return

In [4]:
key_exchange_simulation(128)
print("--------------------------------------------------------------")
key_exchange_simulation(256)

Using 128 bits for p:, p is 258408697565687747289143526799030368827 , q is 129204348782843873644571763399515184413 , generator g is  2  and  Finite Field of size 258408697565687747289143526799030368827
Alice 's secret key is 181566805666945962775574922216486904214  calculates and sends theirs public key: 160215544445620542912893269939232382781
Bob 's secret key b is 246377943868219501211405855951982089032  calculates and sends theirs public key: 34567481099860927979896351899534790139
Is the secret that both sides calculate the same value? True
--------------------------------------------------------------
Using 256 bits for p:, p is 104341851249323468946004203102160229878004685199523181480438502883788288697519 , q is 52170925624661734473002101551080114939002342599761590740219251441894144348759 , generator g is  104341851249323468946004203102160229878004685199523181480438502883788288697517  and  Finite Field of size 10434185124932346894600420310216022987800468519952318148043850288378828

H *man_in_the_middle_simulation(bitsize)* παρουσιάζει πως πετυχαίνει η Επίθεση Ενδιάμεσης Οντότητας:

In [5]:
def man_in_the_middle_simulation(bitsize):
    
    # parameters
    p,q,g,F = generate_parameters(bitsize)
    print( "Using",bitsize,"bits for p:, p is",p,", q is",q,", generator g is ",g," and ",F )

    # Eve's side, sends E to Alice & Bob
    eve = Agent("Eve",p,q,g,F)
    print(eve.name,"'s secret key is",eve.private_key," calculates and sends theirs public key:",eve.public_key)
    E = eve.public_key
    
    # Alice's side, sends A to Eve (thinking it's Bob)
    alice = Agent("Alice",p,q,g,F)
    print(alice.name,"'s secret key is",alice.private_key," calculates and sends theirs public key:",alice.public_key)
    A = alice.public_key

    # Bob's side, sends B to Eve (thinking it's Alice) 
    bob = Agent("Bob",p,q,g,F)
    print(bob.name,"'s secret key b is",bob.private_key," calculates and sends theirs public key:",bob.public_key)
    B = bob.public_key

    #Calculate common secret

    #Alice, knows A,a and E, and thinks she creates session with Bob
    alice.generate_session("Bob",E)

    #Bob, knows B,b and E, and thinks she creates session with Bob
    bob.generate_session("Alice",E)
    #Eve, as Man In The Middle, creates session with both other agents
    eve.generate_session("Alice",A)
    eve.generate_session("Bob",B)


    print("Is the secret that both (Alice & Bob) sides calculate the same value?",alice.sessions[0][1] == bob.sessions[0][1])
    print("Did Eve create two sessions?",len(eve.sessions) == 2)
    print("With whom each one?")
    for s in eve.sessions:
        print(s[0])


Η υλοποίηση της *generate_session(self,agent_name,agent_public_key)* στη κλάση *Agent* βοηθάει λίγο παραπάνω στη κατανόηση για το ποιος ξέρει τί, αλλά και στη περίπτωση της Επίθεσης Ενδιάμεσης Οντότητας, να φανεί πως η Alice και ο Bob προχωρούν νομίζοντας πως έχουν εγκαθιδρύσει συνεδρίας μεταξύ τους, κάτι που δεν ισχύει.

In [6]:
man_in_the_middle_simulation(128)

Using 128 bits for p:, p is 567139692316324716760919544375713246327 , q is 283569846158162358380459772187856623163 , generator g is  567139692316324716760919544375713246325  and  Finite Field of size 567139692316324716760919544375713246327
Eve 's secret key is 550439534011665456077624904476245732426  calculates and sends theirs public key: 27323253666696759353106133176611153782
Alice 's secret key is 371698239318048713418756042572189849531  calculates and sends theirs public key: 274364611385834662668775922498780610527
Bob 's secret key b is 440780834436568545431301173203771442170  calculates and sends theirs public key: 306964882269283359488378618674616509431
Is the secret that both (Alice & Bob) sides calculate the same value? False
Did Eve create two sessions? True
With whom each one?
Alice
Bob
